In [1]:
import os
os.getcwd()

'd:\\Machine Learning\\Deep Learning\\DeepClassifier\\CNN-Classifier\\research'

In [2]:
os.chdir('../')

In [3]:
os.getcwd()

'd:\\Machine Learning\\Deep Learning\\DeepClassifier\\CNN-Classifier'

In [4]:
from collections import namedtuple

In [5]:
DataIngestionConfig = namedtuple("DataIngestionConfig",["root_dir","source_URL","local_data_file","unzipped_dir"])

In [1]:
# Also below can be used in place of namedtuple
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzipped_dir: Path

In [6]:
from deepclassifier.utils import *
from deepclassifier.constants import *

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(root_dir=config.root_dir,
                                                    source_URL=config.source_URL,
                                                    local_data_file=config.local_data_file,
                                                    unzipped_dir=config.unzipped_dir)
        
        return data_ingestion_config
         

In [8]:
import urllib.request as request
from zipfile import ZipFile

class DataIngestion():
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(url=self.config.source_URL,filename=self.config.local_data_file)

    def _get_updated_list_of_files(self,list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(member=f, path=working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzipped_dir)
            

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2024-08-04 13:30:34,119: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2024-08-04 13:30:34,131: INFO: common]: yaml file: params.yaml loaded successfully
[2024-08-04 13:30:34,136: INFO: common]: created directory at: artifacts
[2024-08-04 13:30:34,178: INFO: common]: created directory at: artifacts/data_ingestion
